### 카테고리별 지원 파싱 (취업지원)
취업지원 (68~84페이지) 파싱

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [2]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [3]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [4]:
def parse_employment_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """취업지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "취업 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "국민취업지원제도", "청년구직활동지원금")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["유형", "지원금액", "지원기간"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["1유형", "50만원", "6개월"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 취업지원제도명만
- 대상: 연령, 소득기준, 취업상태 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """취업지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "취업 지원" 제외
- 실제 취업지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "취업지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [5]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(68, 85))  # 취업지원 섹션 (68~84페이지)
employment_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_employment_support(page_text, page_num)
    
    if program:
        employment_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f"{program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 68 파싱 중... (1/17)
실업급여 추출 완료 (표 0개)
페이지 69 파싱 중... (2/17)
국민연금 실업크레딧 추출 완료 (표 0개)
페이지 70 파싱 중... (3/17)
국민취업지원제도 추출 완료 (표 1개)
페이지 71 파싱 중... (4/17)
국민내일배움카드제 추출 완료 (표 1개)
페이지 72 파싱 중... (5/17)
허그일자리지원 프로그램 추출 완료 (표 1개)
페이지 73 파싱 중... (6/17)
취업지원 추출 완료 (표 0개)
페이지 74 파싱 중... (7/17)
자산형성지원 프로그램 추출 완료 (표 2개)
페이지 75 파싱 중... (8/17)
청년내일저축계좌 추출 완료 (표 0개)
페이지 76 파싱 중... (9/17)
근로장려금 추출 완료 (표 1개)
페이지 77 파싱 중... (10/17)
자녀장려금 추출 완료 (표 0개)
페이지 78 파싱 중... (11/17)
여성새로일하기센터 추출 완료 (표 1개)
페이지 79 파싱 중... (12/17)
여성기업종합지원센터 추출 완료 (표 0개)
페이지 80 파싱 중... (13/17)
여성기업 확인서 발급 추출 완료 (표 0개)
페이지 81 파싱 중... (14/17)
여성창업액셀러레이팅 추출 완료 (표 1개)
페이지 82 파싱 중... (15/17)
고령자 계속고용장려금 추출 완료 (표 0개)
페이지 83 파싱 중... (16/17)
고령자 고용지원금 추출 완료 (표 0개)
페이지 84 파싱 중... (17/17)
취업지원 추출 완료 (표 0개)


In [6]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/취업지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(employment_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(employment_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 17개 프로그램
저장 위치: ../../data/outputs/취업지원_2025.json


In [8]:
for program in employment_programs:
    table_count = len(program.get('tables', []))
    print(f"\n{program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    print(f"대상: {program['target'][:80]}...")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


실업급여 (페이지 68, 표 0개)
대상: 회사를 그만두기 이전 18개월 동안 180일 이상 고용보험에 가입하여 근무하고, 본인이 원하지 않았음에도 퇴사한 후 재취업을 위해 적극적으로 노...
내용: 이직 당시 나이와 고용보험 가입기간에 따라 120~270일 간 이직 전 평균임금 60%의 구직급여 지급. 1일 상한액은 6만 6,000원, 1일...

국민연금 실업크레딧 (페이지 69, 표 0개)
대상: 국민연금 가입이력이 1개월 이상 있는 18세 이상 60세 미만 구직급여 수급자 (재산세 과세표준 6억 원 초과, 사업·근로소득 제외한 종합소득금...
내용: 국민연금보험료의 25%를 본인이 부담할 경우 정부에서 최대 12개월까지 지원. 인정소득의 9% 금액으로 산정된 연금보험료를 75%까지 지원. 구...

국민취업지원제도 (페이지 70, 표 1개)
대상: 15~69세, 중위소득 60% 이하, 가구원 재산 4억원 이하...
내용: 국민취업지원제도는 취업활동계획 수립, 구직활동 지원, 소득지원 등을 통해 취업을 지원하는 프로그램입니다. 취업활동계획에 따라 직업훈련, 일경험,...
표: ['유형', '세부 지원내용']...

국민내일배움카드제 (페이지 71, 표 1개)
대상: 국민 누구나 신청 가능. 단, 공무원, 사립학교 교직원, 연매출 4억 원 이상의 자영업자, 월 임금 300만 원 이상의 대규모기업 근로자(만 4...
내용: 국민내일배움카드제는 훈련비 지원율에 따라 일반실업자 등, 참여자, 근로장려금 수급자, 특화훈련 등 다양한 유형으로 지원됩니다. 훈련비 전액 지원...
표: ['구분', '훈련비 지원율']...

허그일자리지원 프로그램 (페이지 72, 표 1개)
대상: 만 15세 이상 ~ 만 69세 이하의 형사처분 또는 보호처분을 받은 사람으로 취업 희망자...
내용: 통합 취업지원프로그램으로, 취업설계, 직업능력개발, 취업성공, 사후관리 등 4단계로 구성되어 있습니다. 각 단계별로 맞춤형 취업지원과 수당이 제...
표: ['취업단계', '내용', '지원